In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from tensorflow import keras

import pickle

pickle_in = open("X_image.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y_image.pickle","rb")
y = pickle.load(pickle_in)

X = X.astype("float32")/255.0

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        
        model = Sequential()
        
        # Convolutional 1
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_1',
                    values=[16, 32, 64],
                    default=32,
                ),
                activation='relu',
                input_shape=X.shape[1:],
                kernel_size=3
            )
        )
        
        # Convolutional 2
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_2',
                    values=[16, 32, 64],
                    default=32,
                ),
                activation='relu',
                kernel_size=3
            )
        )

        # Pooling 1
        model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        
        # Dropout 1
        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.3,
                default=0.1,
                step=0.05,
            ))
        )
        
        # Convolutional 3
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_3',
                    values=[16, 32, 64],
                    default=64,
                ),
                activation='relu',
                kernel_size=3
            )
        )
        
        # Convolutional 4
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_4',
                    values=[16, 32, 64],
                    default=64,
                ),
                activation='relu',
                kernel_size=3
            )
        )

        # Pooling 2
        model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        
        # Dropout 2
        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.3,
                default=0.1,
                step=0.05,
            ))
        )
        
        # Flatten
        model.add(Flatten())
        
        # Hidden layer 1
        model.add(
            Dense(
                units=hp.Int(
                    'units_1',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_1',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 4
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_4',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 2
        model.add(
            Dense(
                units=hp.Int(
                    'units_2',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_2',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 5
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_5',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
            )),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
NUM_CLASSES = 3  # happy neutral sad number of classes
INPUT_SHAPE = (48, 48, 1) 
SEED = 1
HYPERBAND_MAX_EPOCHS = 15
MAX_TRIALS = 40
EXECUTION_PER_TRIAL = 2

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search',
    project_name='version3'
)

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='version3'
)

N_EPOCH_SEARCH = 15
BATCH_SIZE = 8

tuner.search(X, y, batch_size=BATCH_SIZE, epochs=N_EPOCH_SEARCH, validation_split=0.2)

tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

Trial 30 Complete [00h 02m 50s]
val_accuracy: 0.9969696998596191

Best val_accuracy So Far: 0.9969696998596191
Total elapsed time: 01h 01m 41s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in hyperband\version3
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Cov_filters_1: 64
Cov_filters_2: 64
dropout_1: 0.05
Cov_filters_3: 16
Cov_filters_4: 32
dropout_2: 0.30000000000000004
units_1: 416
dense_activation_1: tanh
dropout_4: 0.0
units_2: 160
dense_activation_2: tanh
dropout_5: 0.4
learning_rate: 0.0003021384939866327
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: e8437f10298ae80bb647df59f9a1735a
Score: 0.9969696998596191
Trial summary
Hyperparameters:
Cov_filters_1: 32
Cov_filters_2: 32
dropout_1: 0.2
Cov_filters_3: 64
Cov_filters_4: 32
dropout_2: 0.30000000000000004
units_1: 128
dense_activation_1: sigmoid
dropout_4: 0.0
units_2: 448
dense_activation_2: tanh
dropout_5: 0.5
lear